This is a notebook for testing the "tesnor_tools.py" functionalities.
Author: Suhan | suhan.n.shetty@gmail.com | Date: 10th Sept 2019

In [1]:
import numpy as np
from numpy.linalg import multi_dot as mat_mul
from numpy.random import random_sample 

from tensor_tools import array3

# Specify the tensor
ar3 = array3()
np_shape = [5,3,2]
tensor_shape = [3,2,5] # [mode_1, mode_2 mode_3]: [rows, columns, tubes]

# Test convergence of array normal

# Choose covariance matrices
A = [np.random.randn(tensor_shape[k],tensor_shape[k]) for k in range(3)]
Cov = [mat_mul([A[k],A[k].T]) for k in range(3)]


M = 10+0.00001*np.random.randn(*np_shape)
# Generate data
tensor_data = [ar3.anormal_sampling(M,Cov) for k in range(10)]

# Fit array-normal model
(M_,Cov_,A_) = ar3.anormal(tensor_data, coef=0, constraint=False)
vCov_ = ar3.kron(Cov_)
vCov = ar3.kron(Cov)
# print("Actual")
# print(M)
# print([Cov[j] for j in range(3)])

# print("Estimation: ")
# print(M_)

error_M = np.linalg.norm(M-M_)

#error_cov = [np.linalg.norm(Cov[j]-Cov_[j]) for j in range(3)]
print(error_M)

#rint(error_cov)

print("vCov error: ", np.linalg.norm(vCov-vCov_))

MLN MLE Converged in  9  iterations
11.847607324882222
vCov error:  43.439067298531995


In [2]:
# PARAFAC/CP

# Data
X = ar3.random_array3(tensor_shape)

R = 5 #Number of rank-1 components in the CP

#(A,G) = ar3.cp(X,R) #CP decompostion A = [[a1,a2,..,aR], [b1,b2,...,bR],[c1,c2,...,cR]]
A = ar3.cp(X,R)

# D_ = [np.diag(G[i].flat) for i in range(3)]
# D = mat_mul([D_[0],D_[1],D_[2]])#np.identity(R)#

X_apprx1 = mat_mul([A[0],(ar3.khatri_rao([A[2],A[1]])).T])

Error1 = ar3.unfold(X,1) - X_apprx1

print("Norm of the Residual: ", np.linalg.norm(Error1))

print("Norm of the Data: ", np.linalg.norm(X))
p = 0
q = X.shape[0]*X.shape[1]*X.shape[2]
for A_ in A:
    p = p + A_.shape[0]*A_.shape[1]

print("Number of Parameters in the model: ", p)
print("Number of elements in the data: ", q)



The algorithm has converged. Number of iterations in ALS:  2599
Norm of the Residual:  1.9065509318246339
Norm of the Data:  11.888239793017116
Number of Parameters in the model:  50
Number of elements in the data:  30


In [3]:
# Tucker3 Decomposition 

# Data
X = ar3.random_array3(tensor_shape)
rank_ = [3,3,3]

# Fit Tucker model
(A,G) = ar3.tucker3(X,rank_)
X_ = ar3.tucker_product(G,A,[1,2,3])

# Residual Analysis
print("Norm of the residual: ", np.linalg.norm(X-X_))
print("Norm of the tensor X: ",np.linalg.norm(X))

p = 0
for A_ in A:
    p = p + A_.shape[0]*A_.shape[1]
p = p + G.shape[0]*G.shape[1]*G.shape[2]
print("Number of elements in X: ",X.shape[0]*X.shape[1]*X.shape[2])
print("Number of Parameters: ", p)

The algorithm has converged. Number of iterations:  9
Norm of the residual:  0.7442679746604879
Norm of the tensor X:  11.212346467160325
Number of elements in X:  30
Number of Parameters:  46


In [4]:
# Array-Normal Distribution Test

# Generate Data
X = [ar3.random_array3(tensor_shape) for k in range(10)]
# Or use: X = [np.random.randn(*tensor_shape) for k in range(10)]

# Fit array-normal model
(M,Cov,A) = ar3.anormal(X)

# Sampling from the array-normal distribution 
Xs = ar3.anormal_sampling(M,Cov)

# Array-normal Conditioning
Xt = X[0]
slice_ = 3
Ia = [0,1]
X_a = Xt[Ia,:,:]
M_,Cov_ = ar3.anormal_condition(M,Cov,Ia,X_a, slice_)


MLN MLE Converged in  9  iterations
update_cov:  0.6181128382298201
X_a-Ma: 2.3019856832340593
Cov:  [[0.29387657 0.14382146 0.16111365]
 [0.14382146 0.29909866 0.15627299]
 [0.16111365 0.15627299 0.34514097]] 
 [[0.99478114 0.25622962]
 [0.25622962 0.97385289]]
update_M: 1.2060085911494907


In [5]:
# Tensor-Tensor Regression Test

# Generate Data
X = ar3.random_array3(tensor_shape)
Y = X*2 + 0.1

# Fit a regression model and test the fit for different alpha (hyperparameter)
reps = 0
alpha = 0
while reps<3:
    reps = reps+1
    alpha=alpha+0.3
    print("For alpha %s :"%{alpha})
    R_X=[3,3,5]
    R_Y = R_X
    (Ax,Gx,Ay,Gy,W,Py)=ar3.cov_regression(X,Y, alpha, R_X, R_Y)
    y_mag = 0
    err = 0
    for k in range(tensor_shape[2]):
        X_test = X[k,:,:]
        a = mat_mul([W.T, X_test.flatten()])
        Y_test = mat_mul([a,Py])
        y_mag = y_mag + np.linalg.norm(Y_test)
        err = err + np.linalg.norm(Y_test-Y[k,:,:].flatten())

    print("Expected error in regression: ", err/tensor_shape[2])   
    print("Expected norm of Y: ", y_mag/tensor_shape[2])  


For alpha {0.3} :
The algorithm has converged. Number of iterations:  499
Error in input data fit:  2.1660193427749523e-12 and norm of the input data:  15.5445117174261
Expected error in regression:  0.7138741839566426
Expected norm of Y:  13.938008281882768
For alpha {0.6} :


KeyboardInterrupt: 

In [ ]:
## For the following code you need tensorly package from http://tensorly.org/ 
# import tensorly as tl
# from tensorly.decomposition import tucker, parafac, non_negative_tucker
# rank_ = [6,11,4]
# (G_,A_) = tucker(X, ranks=rank_)
# X_ = tl.tucker_to_tensor(G_,A_)
# print(np.linalg.norm(X_-X))

# A_ = parafac(X, rank=R)
# Xa = mat_mul([A_[0],(ar3.khatri_rao([A_[1],A_[2]])).T])
# X_ = tl.unfold(X,0)
# print(np.linalg.norm(X_-Xa))